In [115]:
import pandas as pd
from time import sleep
from tqdm import tqdm
from itertools import compress

In [2]:
def check_YYYY(YYYY):
    is_good = isinstance(YYYY, str) and len(YYYY) == 4 and YYYY.isdigit()
    if(not(is_good)):
       raise Exception("Bad args")
    return(is_good)

In [3]:
def clean_names(old_cols):
    new_cols = [c.lower().replace('#', '') for c in old_cols]
    return(new_cols)

In [43]:
def get_results_collapsed(YYYY):
    check_YYYY(YYYY)
    url = f"https://rusa.org/cgi-bin/resultsearch_PF.pl?regid=&date={YYYY}&type=&dist=&rtid=&esortby=cert&collapse=1"
    print(url)
    dat = pd.read_html(url)
    dat = dat[0]
    
    new_cols = [c.lower().replace('#', '') for c in list(dat.columns)]
    dat.columns = new_cols
    
    # drop the rows 'x nonmembers also finished'
    dat = dat.drop(dat[dat.cert.str.contains('also finished')].index)
    
    # find 'event divider rows' -- the remaining ones where every column is the same
    event_rows = dat.apply(lambda x: len(set(x)) == 1, axis=1)
    dat['event_row'] = event_rows
    dat['event_id'] = dat.event_row.cumsum()
    return(dat)

In [4]:
def get_events(YYYY):
    check_YYYY(YYYY)
    url = f"https://rusa.org/cgi-bin/resultsearch_PF.pl?regid=&date={YYYY}&type=&dist=&rtid=&esortby=cert&collapse=0"
    print(url)
    
    # get versions with and without links
    dat_links = pd.read_html(url, extract_links='all')[0]
    dat = pd.read_html(url)[0]
    
    # break apart tuples to get urls. we'll have columns called 0 and 1
        # TODO: validate it contains Route and select columns
    route_ids = pd.DataFrame(dat_links[('Route', None)].tolist(), 
                             index=dat_links.index)
    event_ids = pd.DataFrame(dat_links[('select', None)].tolist(), 
                             index=dat_links.index)

    # extract ids from link urls in column 1
        # TODO validate regex, in case format changes
    dat['rt_url'] = route_ids[1]
    dat['rtid'] = route_ids[1].str.extract('^.*rtid=(.*)')
    dat['event_url'] = event_ids[1]
    dat['eid'] = event_ids[1].str.extract('^.*eid=(.*)&')
    
    # clean col names
    new_cols = [c.lower() for c in list(dat.columns)]
    dat.columns = new_cols

    # drop first row and the 'select' column
    dat = dat.drop(0, axis=0)
    dat = dat.drop('select', axis=1)

    return(dat)

In [5]:
events = get_events('2018')
events

https://rusa.org/cgi-bin/resultsearch_PF.pl?regid=&date=2018&type=&dist=&rtid=&esortby=cert&collapse=0


,region,club,type,distance,date,route,rt_url,rtid,event_url,eid
1,VA: Tidewater,Tidewater Randonneurs / 946002,RUSAP,100.0,2018/01/01,Suffolk 100K,/cgi-bin/routesearch_PF.pl?rtid=2034,2034,/cgi-bin/resultsearch_PF.pl?eid=8988&esortby=cert,8988
2,CA: Davis,Davis Bike Club / 905014,RUSAP,103.0,2018/01/01,Moskowite Corner 100Km,/cgi-bin/routesearch_PF.pl?rtid=1322,1322,/cgi-bin/resultsearch_PF.pl?eid=9437&esortby=cert,9437
3,CA: Santa Cruz,Santa Cruz Randonneurs / 905106,ACPB,200.0,2018/01/01,Marina 200k,/cgi-bin/routesearch_PF.pl?rtid=2160,2160,/cgi-bin/resultsearch_PF.pl?eid=9304&esortby=cert,9304
4,MO: Kansas City,Kansas City Ultra Cycling / 925001,RUSAP,104.0,2018/01/01,Barley's - Free State 104,/cgi-bin/routesearch_PF.pl?rtid=1953,1953,/cgi-bin/resultsearch_PF.pl?eid=9632&esortby=cert,9632
5,TX: Dallas,Lone Star Randonneurs / 943026,RUSAP,100.0,2018/01/01,The Doctor's Daughter 100,/cgi-bin/routesearch_PF.pl?rtid=1491,1491,/cgi-bin/resultsearch_PF.pl?eid=8914&esortby=cert,8914
...,...,...,...,...,...,...,...,...,...,...
957,NY: Central/Western,Rochester Bicycling Club / 932030,RUSAB,200.0,2018/12/30,200LR_3 Womens Rights Loop,/cgi-bin/routesearch_PF.pl?rtid=1082,1082,/cgi-bin/resultsearch_PF.pl?eid=10678&esortby=...,10678
958,FL: Central,Central Florida Randonneurs / 909062,RUSAB,200.0,2018/12/30,Swamp Rat 200K,/cgi-bin/routesearch_PF.pl?rtid=2008,2008,/cgi-bin/resultsearch_PF.pl?eid=10666&esortby=...,10666
959,CA: Davis,Davis Bike Club / 905014,RUSAB,204.0,2018/12/31,Davis-Auburn-Davis 200Km,/cgi-bin/routesearch_PF.pl?rtid=1162,1162,/cgi-bin/resultsearch_PF.pl?eid=9448&esortby=cert,9448
960,NY: Central/Western,Rochester Bicycling Club / 932030,RUSAB,200.0,2018/12/31,200LR_3 Womens Rights Loop,/cgi-bin/routesearch_PF.pl?rtid=1082,1082,/cgi-bin/resultsearch_PF.pl?eid=10679&esortby=...,10679


In [77]:
def get_event_details_results(eid):
    
    event_url = f"https://rusa.org/cgi-bin/resultsearch_PF.pl?eid={eid}&esortby=cert"
    #print(event_url)
    two_dats = pd.read_html(event_url)
    
    event_details = two_dats[0]
    event_details.columns = clean_names(list(event_details.columns))
    event_details['eid'] = eid

    # warn if we only get one table back
    if(len(two_dats)<2):
        #print(f'Warning: no finisher results for event {eid}!')
        event_results = pd.DataFrame()
    else:
        event_results = two_dats[1]
        event_results.columns = clean_names(list(event_results.columns))
        # drop the rows 'x nonmembers also finished'
        try:
            event_results = event_results.drop(
                event_results[event_results.cert.str.contains('also finished')].index
            )
        except:
            pass
            #print(f"eid {eid} - couldn't drop 'non member' result rows")
        event_results['eid'] = eid
    
    return([event_details,event_results])

In [107]:
events.groupby('type').size()

type
ACPB     557
ACPF      14
RM         7
RUSAB    165
RUSAF     19
RUSAP    199
dtype: int64

In [108]:
non_team_types = ['ACPB', 'RM', 'RUSAB', 'RUSAP']

In [132]:
# list of event ids for the non team event types, for which we'll get results/details
event_ids = events[events.type.isin(non_team_types)].eid.to_list()

In [36]:
e_detail_list = []
e_results_list = []

In [85]:
for e in tqdm(event_ids[650:]):
    #print(e)
    this_d_e = get_event_details_results(e)
    e_detail_list.append(this_d_e[0])
    e_results_list.append(this_d_e[1])
    if(int(e) % 6 == 0):
        #print('Sleeping')
        sleep(1)

  4%|█▌                                        | 12/311 [00:06<02:39,  1.88it/s]

  9%|███▋                                      | 27/311 [00:15<02:11,  2.17it/s]

 18%|███████▌                                  | 56/311 [00:31<02:55,  1.46it/s]

 23%|█████████▊                                | 73/311 [00:42<01:51,  2.13it/s]

 25%|██████████▍                               | 77/311 [00:44<01:35,  2.44it/s]

...

 98%|████████████████████████████████████████ | 304/311 [02:46<00:03,  2.19it/s]

 98%|████████████████████████████████████████▎| 306/311 [02:48<00:03,  1.60it/s]

 99%|████████████████████████████████████████▍| 307/311 [02:48<00:02,  1.81it/s]

100%|████████████████████████████████████████▊| 310/311 [02:50<00:00,  2.25it/s]

100%|█████████████████████████████████████████| 311/311 [02:50<00:00,  1.82it/s]


In [86]:
print(len(e_detail_list), len(e_results_list), len(event_ids))

961 961 961


In [71]:
event_ids[14]

'9133'

In [93]:
event_results = pd.concat(e_results_list)

In [97]:
event_details = pd.concat(e_detail_list)

In [124]:
# erl2 = list(compress(e_results_list, 
#                      events.type.isin(non_team_types)))

In [146]:
event_details.reset_index(inplace=True, drop=True)
event_results.reset_index(inplace=True, drop=True)

In [147]:
event_results.head()

,cert,rusa,name,club / acp code,time,medal,eid
0,RUSA-P13549,7588,"GREEN, Bill",Davis Bike Club / 905014,06:12,Y,9437
1,RUSA-P13550,8883,"GREEN, Anne",Davis Bike Club / 905014,06:12,NaN,9437
2,RUSA-P13551,6328,"ALBRIGHT, Dean",Davis Bike Club / 905014,05:40,NaN,9437
3,RUSA-P13552,7497,"MACY, Kelly",Davis Bike Club / 905014,05:40,NaN,9437
4,RUSA-P13553,6949,"CARLSON, Drew",San Francisco Randonneurs / 905030,05:40,NaN,9437


In [145]:
event_details.head()

,region,club,type,distance,date,finishers,dnf,eid
0,VA: Tidewater,Tidewater Randonneurs / 946002,RUSAP,100.0,2018/01/01,0,NaN,8988
1,CA: Davis,Davis Bike Club / 905014,RUSAP,103.0,2018/01/01,6,0.0,9437
2,CA: Santa Cruz,Santa Cruz Randonneurs / 905106,ACPB,200.0,2018/01/01,1,1.0,9304
3,MO: Kansas City,Kansas City Ultra Cycling / 925001,RUSAP,104.0,2018/01/01,0,NaN,9632
4,TX: Dallas,Lone Star Randonneurs / 943026,RUSAP,100.0,2018/01/01,6,0.0,8914


In [150]:
event_details.to_csv('event_details_2018.csv', index=False)

In [151]:
event_results.to_csv('event_results_2018.csv', index=False)